<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Обычные-модели" data-toc-modified-id="Обычные-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обычные модели</a></span></li><li><span><a href="#Градиентные-модели" data-toc-modified-id="Градиентные-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Градиентные модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>


# Проект для «Викишоп»


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества _F1_ не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять _BERT_ необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец _text_ в нём содержит текст комментария, а _toxic_ — целевой признак.


**Цель** данной работы - решение задачи классификации с целью определения токсичных комментариев.

Для её реализации были выполнены следующие **задачи**:

1. Загрузить и провести предобработку данных;
2. Обучить и провести оптимизацию нескольких моделей, для обеспечения метрики f1 не меньше, чем 0.75.
3. Сделать выводы по проделанной работе.


## Подготовка


Устанавливаем и подключаем необходимые библиотеки


In [1]:
!pip install catboost==1.1.1 -q
!pip install --upgrade scikit-learn --user -q

In [2]:
import time
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import gdown
import nltk
import re

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import (
    LogisticRegression,
    PassiveAggressiveClassifier,
    SGDClassifier,
    RidgeClassifier,
)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from catboost import CatBoostClassifier

Дополнительные надстройки


In [3]:
RND = 555
np.random.seed(RND)
nltk.download(['stopwords', 'wordnet', 'omw-1.4'])
stopwords = list(set(nltk_stopwords.words('english')))
%config InlineBackend.figure_format = 'retina'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vovan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vovan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vovan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Загружаем и изучаем датасет


In [4]:
df = pd.read_csv("https://code.s3.yandex.net/datasets/toxic_comments.csv")
df = df.iloc[:, 1:]
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Рассмотрим распределение меток


In [6]:
df["toxic"].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Классы несбалансированы, токсичных всего 10 %. Проводить апсемплинг данных с таким количеством строк неразумно, поэтому в для корректного построения моделей будем использовать параметр class_weights='balansed'.


In [7]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


Данные требуют обработки, а именно очистки и лемматизации для дальнейшего упрощения моделирования.


In [8]:
def clean_text(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub("\W", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\d+", " ", text)
    text = text.strip(" ")
    return text

Применяем очистку и лемматизацию, исключая ненужные слова


In [9]:
%%time
wnl = WordNetLemmatizer()
df['lemm_text'] = df['text'].str.lower().map(lambda sen: clean_text(sen))
df['lemm_text'] = df['lemm_text'].str.split().apply(lambda sen: ' '.join(
    [wnl.lemmatize(word) for word in sen if word not in stopwords]))

CPU times: total: 1min 15s
Wall time: 1min 15s


Проверим эффективность предобработки на одной из строк


In [10]:
display(df["text"][4])
display(df["lemm_text"][4])

"You, sir, are my hero. Any chance you remember what page that's on?"

'sir hero chance remember page'

In [11]:
df[["text", "lemm_text", "toxic"]]

,text,lemm_text,toxic
0,Explanation\nWhy the edits made under my usern...,explanation edits made username hardcore metal...,0
1,D'aww! He matches this background colour I'm s...,aww match background colour seemingly stuck th...,0
2,"Hey man, I'm really not trying to edit war. It...",hey man really trying edit war guy constantly ...,0
3,"""\nMore\nI can't make any real suggestions on ...",cannot make real suggestion improvement wonder...,0
4,"You, sir, are my hero. Any chance you remember...",sir hero chance remember page,0
...,...,...,...
159287,""":::::And for the second time of asking, when ...",second time asking view completely contradicts...,0
159288,You should be ashamed of yourself \n\nThat is ...,ashamed horrible thing put talk page,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",spitzer umm there actual article prostitution ...,0
159290,And it looks like it was actually you who put ...,look like actually put speedy first version de...,0


### Вывод

1. Текстовые данные загружены и содержат примерно 160_000 строк;
2. Тексты очищены и лемматизированы с помощью инструментов библиотек re и nltk.
3. В данных наблюдается дисбаланс классов, токсичных комментариев порядка 10 % всего датасета;


## Обучение


Разбиваем данные на обучающую, валидационную и тестовую выборки в соотношении 60/20/20


In [12]:
X = df["lemm_text"]
y = df["toxic"]
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, random_state=RND, test_size=0.2
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, random_state=RND, test_size=0.25
)
print(X_train.shape, X_val.shape, X_test.shape)

(95574,) (31859,) (31859,)


Проверим дисбаланс классов в получившихся выборках


In [13]:
pd.DataFrame(
    {
        "y_test": y_test.value_counts(normalize=True),
        "y_val": y_val.value_counts(normalize=True),
        "y_train": y_train.value_counts(normalize=True),
    }
)

,y_test,y_val,y_train
0,0.897015,0.898208,0.898906
1,0.102985,0.101792,0.101094


### Обычные модели


Выберем 5 моделей из библиотеки slearn обучим их и запишем необходимые метрики качества моделей, чтобы в дальнейшем решить, какие из них имеет смысл оптимизировать. Функция test_models, работает в 2 режимах: True необходим для примерной оценки времени обучения и предсказания модели на валидации и оценки кросс-валидации только на обучающих данных, False оценивает данные на тесте, но обучающая выборка состоит из train+val. Фичи из текста для моделей создаёт TfidfVectorizer, который остеивает очень частые и очень редкие слова, а также стоп-слова.


In [14]:
default_params = {"random_state": RND, "class_weight": "balanced"}
tf_idf = TfidfVectorizer(
    ngram_range=(1, 3), min_df=5, max_df=0.8, sublinear_tf=True, stop_words="english"
)
classifiers = [
    LogisticRegression(**default_params),
    LinearSVC(**default_params),
    SGDClassifier(
        loss="log_loss",
        max_iter=1000,
        penalty="l2",
        alpha=1e-5,
        tol=1e-3,
        **default_params
    ),
    PassiveAggressiveClassifier(**default_params),
    RidgeClassifier(**default_params),
]

In [15]:
def test_models(classifiers, train_mode=True):
    results = []
    for clf in classifiers:
        clf_name = clf.__class__.__name__
        print("=" * 80)
        print(f"Training {clf_name}...\n")
        pipe = (
            clf
            if clf_name == "CatBoostClassifier"
            else Pipeline(steps=[("tf_idf", tf_idf), ("model", clf)])
        )
        features_train = X_train if train_mode else X_train_full
        if clf_name == "CatBoostClassifier":
            features_train = pd.DataFrame(features_train)
        target_train = y_train if train_mode else y_train_full
        cvs = cross_val_score(
            pipe, features_train, target_train, scoring="f1", n_jobs=-1, cv=3
        )
        print("Train cross-val:", cvs.mean(), ":", cvs)

        fit_start = time.perf_counter()
        pipe.fit(features_train, target_train)
        fit_time = time.perf_counter() - fit_start
        print(f"Fit time: {fit_time:.3f} seconds")

        features_test = X_val if train_mode else X_test
        predict_start = time.perf_counter()
        y_pred = pipe.predict(
            pd.DataFrame(features_test)
            if clf_name == "CatBoostClassifier"
            else features_test
        )
        predict_time = time.perf_counter() - predict_start
        print(f"Prediction time: {predict_time:.3f} seconds")

        y_true = y_val if train_mode else y_test
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        roc_auc = roc_auc_score(y_true, y_pred)

        print(
            f"""
        Metrics:
        {clf_name} accuracy: {accuracy:.3f}
        {clf_name} f1 score: {f1:.3f}
        {clf_name} ROC AUC score: {roc_auc:.3f}
        """
        )
        print("=" * 80)
        results.append(
            [clf_name, fit_time, predict_time, cvs.mean(), accuracy, f1, roc_auc, pipe]
        )
    return results


sklearn_clfs = test_models(classifiers, train_mode=True)

Training LogisticRegression...

Train cross-val: 0.7369407295314486 : [0.74031764 0.73363304 0.73687151]
Fit time: 17.444 seconds
Prediction time: 3.398 seconds

        Metrics:
        LogisticRegression accuracy: 0.945
        LogisticRegression f1 score: 0.758
        LogisticRegression ROC AUC score: 0.902
        
Training LinearSVC...

Train cross-val: 0.744282890007129 : [0.74372307 0.74320693 0.74591867]
Fit time: 15.888 seconds
Prediction time: 3.367 seconds

        Metrics:
        LinearSVC accuracy: 0.949
        LinearSVC f1 score: 0.761
        LinearSVC ROC AUC score: 0.882
        
Training SGDClassifier...

Train cross-val: 0.7392924402672728 : [0.74592009 0.73318572 0.73877151]
Fit time: 14.857 seconds
Prediction time: 3.205 seconds

        Metrics:
        SGDClassifier accuracy: 0.944
        SGDClassifier f1 score: 0.755
        SGDClassifier ROC AUC score: 0.904
        
Training PassiveAggressiveClassifier...

Train cross-val: 0.7168529160198819 : [0.71484559 

Для более удобного отображения напишем функцию, создающую из списка датафрейм.


In [16]:
def df_results(results):
    columns = [
        "model_name",
        "fit_time",
        "pred_time",
        "cross_val_score_f1",
        "accuracy",
        "f1",
        "AUC_ROC",
        "model",
    ]
    df = (
        pd.DataFrame(results, columns=columns)
        .drop("model", axis=1)
        .sort_values(by="cross_val_score_f1", ascending=False)
    )
    return df

In [17]:
df_results(sklearn_clfs).style.background_gradient("PuBu")

,model_name,fit_time,pred_time,cross_val_score_f1,accuracy,f1,AUC_ROC
1,LinearSVC,15.887598,3.366891,0.744283,0.949057,0.761007,0.881553
2,SGDClassifier,14.857276,3.205218,0.739292,0.943501,0.754835,0.904024
0,LogisticRegression,17.444447,3.397604,0.736941,0.944725,0.757738,0.902382
3,PassiveAggressiveClassifier,16.666684,3.107000,0.716853,0.948052,0.737593,0.845724
4,RidgeClassifier,16.250820,3.408354,0.686832,0.934461,0.699222,0.851965


Судя по результатам на кросс-валидации, 4 модели можно оптимизировать дальше, чтобы улучшить метрику. Первые 3 модели показывают хорошие результаты по всем 3 метрикам.


Пробуем оптимизировать эти модели со следующими параметрами:

1. LinearSVC: _penalty_ = 'l1'/'l2', _C_ = 1e-6 - 1e4, _dual_ = True/False;
2. SGDClassifier: _loss_ = 'hinge'/'log'/'modified*huber'/'squared_hinge'/'perceptron'/'huber'/'epsilon_insensitive', \_penalty* = 'l1'/'l2', _alpha_ = 1e-8 - 1e4, _validation_fraction_ = 0.01 - 0.99;
3. LogisticRegression: _penalty_ = 'l1'/'l2', _C_ = 1e-4 - 1e4, _solver_ = 'lbfgs'/'liblinear'/'newton-cg'/'sag';
4. PassiveAgressiveClassifier: _validation_fraction_ = 0.01 - 0.99, _loss_ = 'squared*hinge'/'hinge', \_average* = 0 - 100, _C_ = 1e-6 - 1e4.

Поскольку процесс оптимизации рассчитывался достаточно долго, то я не буду приводить код для каждой модели, а ограничусь лишь общим описанием процесса оптимизации на примере SGDClassifier:

```python
def objective(trial):
    pipe = Pipeline(steps=[('tf_idf', tf_idf), ('model', SGDClassifier(**default_params, eta0=0.1, learning_rate='adaptive', early_stopping=True))])

    loss = trial.suggest_categorical('loss', ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'huber', 'epsilon_insensitive'])
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    alpha = trial.suggest_float('alpha', 1e-8, 1e4, log=True)
    validation_fraction = trial.suggest_float('validation_fraction', 0.01, 0.99, step=0.01)

    pipe.set_params(model__loss=loss, model__penalty=penalty, model__alpha=alpha, model__validation_fraction=validation_fraction)

    return cross_val_score(pipe, X_train_full, y_train_full, cv=3, scoring='f1', n_jobs=3).mean()

study_sgd_new = optuna.create_study(study_name='new_sgd', direction='maximize')
study_sgd_new.optimize(objective, n_trials=100, n_jobs=9)
```

Далее оптимизированные функции прогонялись через test_models в режиме train_mode=False и результаты архивировались в файл .pkl через pickle.


In [18]:
# Ячейка не выполнится и даст ошибку, если в папке с проектом не будет файла modified_models_list.pkl, ссылки для скачивания в начале
modified_models_list = pickle.load(open("modified_models_list.pkl", "rb"))
df_results(modified_models_list[:-1]).iloc[:, :4].style.background_gradient("PuBu")

,model_name,fit_time,pred_time,cross_val_score_f1
2,SGDClassifier,19.681963,2.756154,0.765352
0,LogisticRegression,25.002347,2.972585,0.760968
3,PassiveAggressiveClassifier,17.499214,2.717646,0.759005
1,LinearSVC,17.357877,2.645502,0.757993


### Градиентные модели


В качестве градиетных моделей для данной задачи были выбраны CatBoost и LGBM классификаторы, в силу того, что CatBoost имеет поддержку обработки текстовых данных(если выполнять классификацию в обычном режиме через предложенный в test_models pipeline, то на одну итерацию приходится ~15 мин), LGBM же должен за более разумное время справляться с такими массивами данных(после tf_idf размер данных примерно 120_000x145_000).


Создаём CatBoost классификатор и добавляем его к остальным сохранённым моделям


In [19]:
%%time
clf_cb = CatBoostClassifier(
    learning_rate=0.33, text_features=['lemm_text'], eval_metric='F1', 
    early_stopping_rounds=100, random_seed=RND, verbose=100)
all_clfs_results = modified_models_list + test_models([clf_cb], False)

Training CatBoostClassifier...

Train cross-val: 0.7673910594510046 : [0.76889177 0.77052795 0.76275345]
0:	learn: 0.6940104	total: 303ms	remaining: 5m 2s
100:	learn: 0.7779708	total: 20.7s	remaining: 3m 3s
200:	learn: 0.7984593	total: 39.7s	remaining: 2m 37s
300:	learn: 0.8091379	total: 58.4s	remaining: 2m 15s
400:	learn: 0.8190836	total: 1m 17s	remaining: 1m 55s
500:	learn: 0.8270111	total: 1m 35s	remaining: 1m 35s
600:	learn: 0.8349812	total: 1m 53s	remaining: 1m 15s
700:	learn: 0.8424224	total: 2m 12s	remaining: 56.5s
800:	learn: 0.8514592	total: 2m 35s	remaining: 38.7s
900:	learn: 0.8583445	total: 2m 54s	remaining: 19.1s
999:	learn: 0.8649872	total: 3m 13s	remaining: 0us
Fit time: 199.625 seconds
Prediction time: 0.591 seconds

        Metrics:
        CatBoostClassifier accuracy: 0.958
        CatBoostClassifier f1 score: 0.773
        CatBoostClassifier ROC AUC score: 0.841
        
CPU times: total: 25min 54s
Wall time: 10min 16s


Для конечного выбора модели необходимо скрыть значения метрик на тестовой выборке, в таком случае определяющими параметрами для выбора итогового классификатора будут как и ранее время обучения, предсказания и значение f1 на полной обучающей выборке с кросс-валидацией.


In [20]:
df_clfs_results = df_results(all_clfs_results)
df_clfs_results.iloc[:, :4].style.background_gradient("PuBu")

,model_name,fit_time,pred_time,cross_val_score_f1
4,LGBMClassifier,708.931170,7.029682,0.772423
5,CatBoostClassifier,199.624616,0.590831,0.767391
2,SGDClassifier,19.681963,2.756154,0.765352
0,LogisticRegression,25.002347,2.972585,0.760968
3,PassiveAggressiveClassifier,17.499214,2.717646,0.759005
1,LinearSVC,17.357877,2.645502,0.757993


Комментарии по результатам:

1. Скорость обучения - в плане скорости LGBM и CatBoost уступают моделям из sklearn примерно в 35+ и в 7+ раз;
2. Скорость предсказания - CatBoost оказался самым быстрым, далее идут sklearn модели с результатом <3 сек и на последнем месте LGBM;
3. F1 метрика на кросс-валидации - градиентные модели самые эффективные (дают метрику >0.765), остальные же модели несколько уступают и дают на кроссвалидации от 0.758 до 0.761.

В рамках данной задачи компромисным вариантом среди всех предложенных я считаю SGDClassifier, поскольку при соответствующей подготовке и данных и подборе гиперпараметров способен дать метрику не хуже, чем более солидные модели. В качестве альтернативного варианта я бы рассмотрел CatBoost, который при минимальной настройке даёт хорошие метрики и имеет встроенную возможность работы с текстом.


In [21]:
df_clfs_results.query('model_name == "SGDClassifier"')

,model_name,fit_time,pred_time,cross_val_score_f1,accuracy,f1,AUC_ROC
2,SGDClassifier,19.681963,2.756154,0.765352,0.95138,0.7715,0.883058


Как видим, этот классификатор даёт метрику f1 > 0.75. Также следует отметить, что модель на тестовых данных адекватна, поскольку в случае предсказания наиболее частого класса(в данном случае 0) метрика f1, будет равна 0.


### Вывод

1. Рассмотрено 6 различных классификатора, 4 из библиотеки sklearn, и 2 сторонние модели градиентого бустинга: LGBMClassifier и CatBoostClassifier;
2. Для всех моделей, кроме CatBoostClassifier были подобраны соответствующие гиперпараметры, позволившие увеличить метрику f1 до значения большего, чем 0.75.
3. Оптимальной для данной задачи можно считать SGDClassifier.


## Выводы


По результатам данной работы можно сделать следующие выводы:

1. Тестовые данные были загружены, очищены и лемматизированы;
2. Обучено 6 различных моделей, 2 из которых представляют собой мощные градиентные бустинги, для большинства моделей гиперпараметры были оптимизированы для достижения метрики f1 > 0.75.
3. Для дальнейшего использования рекомендована модель SGDClassifier, поскольку она адекватна по скорости обучения и итоговым метрикам. В случае, если необходимо свести подготовку данных к минимуму, стоит рассмотреть возможность использования CatBoostClassifier с активным параметром text_features.


## Чек-лист проверки


- [x] Jupyter Notebook открыт
- [x] Весь код выполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Данные загружены и подготовлены
- [x] Модели обучены
- [x] Значение метрики _F1_ не меньше 0.75
- [x] Выводы написаны
